<a href="https://colab.research.google.com/github/Infant-Joshva/Data-Science-learning-path/blob/main/Project_3(Employee_Attrition_ML_Model)Git/ML_Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model selection

#### ETL

In [ ]:
# Importing packages for data loading

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Raw GitHub cleaaned dataset(CSV) link
url = "https://raw.githubusercontent.com/Infant-Joshva/Data-Science-learning-path/refs/heads/main/Project_3(Employee_Attrition_ML_Model)Git/data%20set/cleaned_employee_data.csv"

emp_df = pd.read_csv(url).reset_index(drop=True)
emp_df.head()

#### Preprocessing Pipeline

In [ ]:
# Importing lib for Pipeline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# emp_df['Attrition_binary'] = emp_df['Attrition'].map({'Yes': 1, 'No': 0})

In [ ]:
# Creating Features and Target in new DataFrame

X=emp_df[['Age','DistanceFromHome','Education','EnvironmentSatisfaction','JobInvolvement','JobLevel','JobSatisfaction','MonthlyIncome','NumCompaniesWorked','PercentSalaryHike','PerformanceRating','RelationshipSatisfaction',
         'TotalWorkingYears','TrainingTimesLastYear','WorkLifeBalance','YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager','BusinessTravel',
         'Department','EducationField','Gender','JobRole','MaritalStatus','OverTime']] # Features
Y=emp_df['Attrition'] # Target

In [ ]:
# Seperating Numerical and Categorical Features
# For 1st we seperating Numerical into 2 types one is Continues and Discreat

num_feature=X[X.select_dtypes(include=['int64','float64','int32']).columns]
num_feature

In [ ]:
# We have continues and discret values so we again seperating numerical values
num_feature_dis = num_feature[['Education','EnvironmentSatisfaction','JobInvolvement','JobLevel','JobSatisfaction',
                               'RelationshipSatisfaction','WorkLifeBalance','PerformanceRating']].reset_index(drop=True) # Discreat Values
                               #Need to confirm "StockOptionLevel" is needed or not
num_features_con = num_feature[['Age','DistanceFromHome','MonthlyIncome','NumCompaniesWorked','PercentSalaryHike','TotalWorkingYears','TrainingTimesLastYear',
                                'YearsAtCompany','YearsInCurrentRole','YearsSinceLastPromotion','YearsWithCurrManager']].reset_index(drop=True) # Continues Values

In [ ]:
# Seperating Categorical features

cat_feature=X[X.select_dtypes(include=['object']).columns].reset_index(drop=True)
cat_feature

In [ ]:
# Seperating Categorical features for OneHot Encoding

cat_feature_OH = cat_feature[['BusinessTravel','Department','Gender','MaritalStatus','OverTime']].reset_index(drop=True)
cat_feature_OH

In [ ]:
# Seperating Categorical features for Target Encoding

cat_feature_TR = cat_feature[['EducationField','JobRole']].reset_index(drop=True)
cat_feature_TR

In [ ]:
cat_feature_TR_cols = cat_feature_TR.columns
cat_feature_TR_cols

In [ ]:
# Loop through each categorical column

# This will store the results
cat_feature_TR_cols_cross_list = []

for col in cat_feature_TR_cols:
    crosstab = pd.crosstab(emp_df[col], emp_df['Attrition'])

    # Ensure both 'Yes' and 'No' are present to avoid KeyErrors
    if 'Yes' not in crosstab.columns:
        crosstab['Yes'] = 0
    if 'No' not in crosstab.columns:
        crosstab['No'] = 0


    # Compute the attrition ratio
    crosstab['Attrition_Ratio'] = crosstab['Yes'] / (crosstab['Yes'] + crosstab['No'])

    # Append the sorted index (category names) by attrition ratio
    sorted_index = crosstab.sort_values(by='Attrition_Ratio', ascending=False).index
    cat_feature_TR_cols_cross_list.append(sorted_index)

cat_feature_TR_cols_cross_list

In [ ]:
# The list for Odinal Encoding Categorical value

cat_feature_TR_cols_cross_list

In [ ]:
# Creating transformer for Numerical con columns

numerical_transformer_con=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler())
])

numerical_transformer_con

In [ ]:
# Creating transformer for dis column

numerical_transformer_dis=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent'))
])

numerical_transformer_dis

In [ ]:
# Creating transformer for cat OneHotEncoder column

categorical_transformer_OH=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OneHotEncoder(dtype='int64', sparse_output=False, drop='first'))
])

categorical_transformer_OH

In [ ]:
# Creating transformer for cat odinal column

categorical_transformer_TR=Pipeline(steps=[
    ('imputer',SimpleImputer(strategy='most_frequent')),
    ('encoder',OrdinalEncoder(categories=[['Human Resources', 'Technical Degree', 'Marketing', 'Life Sciences','Medical', 'Other'],
     ['Sales Representative', 'Laboratory Technician', 'Human Resources','Sales Executive', 'Research Scientist', 'Manufacturing Director','Healthcare Representative', 'Manager', 'Research Director']]))
])

categorical_transformer_TR

In [ ]:
# Combining the all transformer as a full loaded pipeline with column transformer

preprocess=ColumnTransformer(
    transformers=[
        ('num_con',numerical_transformer_con,num_features_con.columns),
        ('num_dis',numerical_transformer_dis,num_feature_dis.columns),
        ('cat_OH',categorical_transformer_OH,cat_feature_OH.columns),
        ('cat_TR',categorical_transformer_TR,cat_feature_TR.columns)
    ]
)
preprocess



*   preprocess ready
*   here i'm going to prepare ML model



#### Data Balancing

In [ ]:
from imblearn.pipeline import Pipeline as IMBPipeline

In [ ]:
from imblearn.over_sampling import SMOTE

In [137]:
# Splitting data for train and test

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((1176, 26), (294, 26), (1176,), (294,))

In [138]:
X_train_trans=preprocess.fit_transform(X_train)

In [139]:
smote=SMOTE(random_state=42)
X_train_smote,y_train_smote=smote.fit_resample(X_train_trans,Y_train)
y_train_smote.value_counts()

,count
Attrition,
No,978
Yes,978


#### ML Pipeline

In [ ]:
# Data preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

In [143]:
# Machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

# Evaluation metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report

##### LogisticRegression

In [140]:
# Creating pipeline for LogReg

logreg_pipeline=IMBPipeline(steps=[
    ('preprocess',preprocess),
    ('Sampling',SMOTE(random_state=42)),
    ('Poly',PolynomialFeatures(degree=2)),
    ('logreg',LogisticRegression(class_weight="balanced", C=0.1))
])

logreg_pipeline

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_con',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLast...
                                                                                               'Laboratory '
                                                                                               'Technician',
                                                                                               'Human '
                                                                                               'Resources',
                                                                                               'Sales '
                                                                                               'Executive',
                                                                                               'Research '
                                                                                               'Scientist',
                                                                                               'Manufacturing '
                                                                                               'Director',
                                                                                               'Healthcare '
                                                                                               'Representative',
                                                                                               'Manager',
                                                                                               'Research '
                                                                                               'Director']]))]),
                                                  Index(['EducationField', 'JobRole'], dtype='object'))])),
                ('Sampling', SMOTE(random_state=42)),
                ('Poly', PolynomialFeatures()),
                ('logreg', LogisticRegression(C=0.1, class_weight='balanced'))])

In [141]:
logreg_pipeline.fit(X_train,Y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num_con',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'TotalWorkingYears', 'TrainingTimesLastYear',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLast...
                                                                                               'Laboratory '
                                                                                               'Technician',
                                                                                               'Human '
                                                                                               'Resources',
                                                                                               'Sales '
                                                                                               'Executive',
                                                                                               'Research '
                                                                                               'Scientist',
                                                                                               'Manufacturing '
                                                                                               'Director',
                                                                                               'Healthcare '
                                                                                               'Representative',
                                                                                               'Manager',
                                                                                               'Research '
                                                                                               'Director']]))]),
                                                  Index(['EducationField', 'JobRole'], dtype='object'))])),
                ('Sampling', SMOTE(random_state=42)),
                ('Poly', PolynomialFeatures()),
                ('logreg', LogisticRegression(C=0.1, class_weight='balanced'))])

In [142]:
Y_train_pred=logreg_pipeline.predict(X_train)
Y_test_pred=logreg_pipeline.predict(X_test)

print('Confusion Matrix Train')
print('--'*50)
print(confusion_matrix(Y_train,Y_train_pred))
print('Confusion Matrix Test')
print('--'*50)
print(confusion_matrix(Y_test,Y_test_pred))
print('**'*50)
print('Classification Report Train')
print('--'*50)
print(classification_report(Y_train,Y_train_pred))
print('Classification Report Test')
print('--'*50)
print(classification_report(Y_test,Y_test_pred))
print('--'*50)


Confusion Matrix Train
----------------------------------------------------------------------------------------------------
[[903  75]
 [ 19 179]]
Confusion Matrix Test
----------------------------------------------------------------------------------------------------
[[224  31]
 [ 18  21]]
****************************************************************************************************
Classification Report Train
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

          No       0.98      0.92      0.95       978
         Yes       0.70      0.90      0.79       198

    accuracy                           0.92      1176
   macro avg       0.84      0.91      0.87      1176
weighted avg       0.93      0.92      0.92      1176

Classification Report Test
----------------------------------------------------------------------------------------------------
              precisio

##### SVC

In [ ]:
# Creating pipeline for SVC

svc_pipeline=IMBPipeline(steps=[
    ('preprocess',preprocess),
    ('Sampling',SMOTE(random_state=42)),
    ('svc',SVC(class_weight="balanced", C=0.1))
])

svc_pipeline

In [ ]:
svc_pipeline.fit(X_train,Y_train)

In [ ]:
Y_train_pred=svc_pipeline.predict(X_train)
Y_test_pred=svc_pipeline.predict(X_test)

print('Confusion Matrix Train')
print('--'*50)
print(confusion_matrix(Y_train,Y_train_pred))
print('Confusion Matrix Test')
print('--'*50)
print(confusion_matrix(Y_test,Y_test_pred))
print('**'*50)
print('Classification Report Train')
print('--'*50)
print(classification_report(Y_train,Y_train_pred))
print('Classification Report Test')
print('--'*50)
print(classification_report(Y_test,Y_test_pred))
print('--'*50)


##### DecisionTreeClassifier

In [ ]:
# Creating pipeline for DecisionTreeClassifier

dt_pipeline=IMBPipeline(steps=[
    ('preprocess',preprocess),
    ('Sampling',SMOTE(random_state=42)),
    ('dt',DecisionTreeClassifier(class_weight={'No':1,'Yes':8},max_depth=8,min_samples_split=15,min_samples_leaf=15))
])

dt_pipeline

In [ ]:
dt_pipeline.fit(X_train,Y_train)

In [ ]:
Y_train_pred=dt_pipeline.predict(X_train)
Y_test_pred=dt_pipeline.predict(X_test)

print('Confusion Matrix Train')
print('--'*50)
print(confusion_matrix(Y_train,Y_train_pred))
print('Confusion Matrix Test')
print('--'*50)
print(confusion_matrix(Y_test,Y_test_pred))
print('**'*50)
print('Classification Report Train')
print('--'*50)
print(classification_report(Y_train,Y_train_pred))
print('Classification Report Test')
print('--'*50)
print(classification_report(Y_test,Y_test_pred))
print('--'*50)

##### Random Forest

In [ ]:
# Creating pipeline for Random Forest

RF_pipeline = IMBPipeline(steps=[
    ('preprocess', preprocess),
    ('Sampling', SMOTE(random_state=42)),
    ('rf', RandomForestClassifier(class_weight={'No':4,'Yes':6},max_depth=10,min_samples_split=20,min_samples_leaf=20, n_estimators=500))

])

RF_pipeline

In [ ]:
RF_pipeline.fit(X_train,Y_train)

In [ ]:
Y_train_pred=RF_pipeline.predict(X_train)
Y_test_pred=RF_pipeline.predict(X_test)

print('Confusion Matrix Train')
print('--'*50)
print(confusion_matrix(Y_train,Y_train_pred))
print('Confusion Matrix Test')
print('--'*50)
print(confusion_matrix(Y_test,Y_test_pred))
print('**'*50)
print('Classification Report Train')
print('--'*50)
print(classification_report(Y_train,Y_train_pred))
print('Classification Report Test')
print('--'*50)
print(classification_report(Y_test,Y_test_pred))
print('--'*50)

##### XGBclassifier

In [145]:
Y_train.value_counts()

,count
Attrition,
No,978
Yes,198


In [146]:
len(Y_train[Y_train=='No'])/len(Y_train[Y_train=='Yes'])

4.9393939393939394

In [147]:
class_ratio=len(Y_train[Y_train=='No'])/len(Y_train[Y_train=='Yes'])

In [153]:
from sklearn.preprocessing import LabelEncoder

# encode Y
le = LabelEncoder()
Y_train_enc = le.fit_transform(Y_train)   # No=0, Yes=1
Y_test_enc  = le.transform(Y_test)

XGB_pipeline=IMBPipeline(steps=[
    ('Preprcess',preprocess),
    ('Sampling',SMOTE(random_state=42)),
    ('Model',XGBClassifier(scale_pos_weight=class_ratio, max_depth=4 ))

])

# now train again
XGB_pipeline.fit(X_train, Y_train_enc)

Y_train_pred = XGB_pipeline.predict(X_train)
Y_test_pred  = XGB_pipeline.predict(X_test)

print('Confusion Matrix Train')
print(confusion_matrix(Y_train_enc, Y_train_pred))
print('Confusion Matrix Test')
print(confusion_matrix(Y_test_enc, Y_test_pred))

print('Classification Report Train')
print(classification_report(Y_train_enc, Y_train_pred, target_names=le.classes_))
print('Classification Report Test')
print(classification_report(Y_test_enc, Y_test_pred, target_names=le.classes_))


Confusion Matrix Train
[[978   0]
 [  0 198]]
Confusion Matrix Test
[[226  29]
 [ 22  17]]
Classification Report Train
              precision    recall  f1-score   support

          No       1.00      1.00      1.00       978
         Yes       1.00      1.00      1.00       198

    accuracy                           1.00      1176
   macro avg       1.00      1.00      1.00      1176
weighted avg       1.00      1.00      1.00      1176

Classification Report Test
              precision    recall  f1-score   support

          No       0.91      0.89      0.90       255
         Yes       0.37      0.44      0.40        39

    accuracy                           0.83       294
   macro avg       0.64      0.66      0.65       294
weighted avg       0.84      0.83      0.83       294



##### KNeighborsClassifier

In [159]:
KNeighborsClassifier_pipeline=IMBPipeline(steps=[
    ('preprocess',preprocess),
    ('Sampling',SMOTE(random_state=42)),
    ('knn',KNeighborsClassifier(
    n_neighbors=5,        # Number of nearest neighbors (k value)
    weights='uniform',    # 'uniform' = all neighbors equal, 'distance' = closer neighbors have more influence
    algorithm='auto',     # 'auto', 'ball_tree', 'kd_tree', 'brute'
    leaf_size=30,         # Leaf size for BallTree/KDTree (affects speed)
    p=2,                  # Power parameter for distance: p=1 → Manhattan, p=2 → Euclidean
    metric='minkowski',   # Distance metric (default minkowski with p=2 = Euclidean)
    metric_params=None,   # Extra metric arguments (rarely used)
    n_jobs=None           # Parallel jobs (-1 = use all CPUs)
))
])

KNeighborsClassifier_pipeline.fit(X_train,Y_train)

Y_train_pred=KNeighborsClassifier_pipeline.predict(X_train)
Y_test_pred=KNeighborsClassifier_pipeline.predict(X_test)

print('Confusion Matrix Train')
print('--'*50)
print(confusion_matrix(Y_train,Y_train_pred))
print('Confusion Matrix Test')
print('--'*50)
print(confusion_matrix(Y_test,Y_test_pred))
print('**'*50)
print('Classification Report Train')
print('--'*50)
print(classification_report(Y_train,Y_train_pred))
print('Classification Report Test')
print('--'*50)
print(classification_report(Y_test,Y_test_pred))
print('--'*50)


Confusion Matrix Train
----------------------------------------------------------------------------------------------------
[[762 216]
 [  1 197]]
Confusion Matrix Test
----------------------------------------------------------------------------------------------------
[[178  77]
 [ 14  25]]
****************************************************************************************************
Classification Report Train
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

          No       1.00      0.78      0.88       978
         Yes       0.48      0.99      0.64       198

    accuracy                           0.82      1176
   macro avg       0.74      0.89      0.76      1176
weighted avg       0.91      0.82      0.84      1176

Classification Report Test
----------------------------------------------------------------------------------------------------
              precisio